In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('Elec_Demand_train.csv')[:1000]

demand_set = data.iloc[:, 1].values
date_set = data.iloc[:, 0].values

In [3]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(demand_set)

In [5]:
hours = [index % 24 for index, _ in enumerate(date_set)]

In [12]:
demand_X = []
hour_X = []
y = []

for i in range(60, len(demand_set)):
    demand_X.append(training_set_scaled[i-60:i])
    hour_X.append(hours[i-60:i])
    y.append(training_set_scaled[i, 0])

demand_X, hour_X, y = np.array(demand_X), np.array(hour_X), np.array(y)
hour_X = np.reshape(hour_X, (hour_X.shape[0], hour_X.shape[1], 1))
print(demand_X.shape)
print(hour_X.shape)

(940, 60, 1)
(940, 60, 1)


In [13]:
demand_hours_X = np.hstack((demand_X, hour_X))
demand_hours_X

array([[[ 0.2909783 ],
        [ 0.25874636],
        [ 0.23769031],
        ...,
        [ 9.        ],
        [10.        ],
        [11.        ]],

       [[ 0.25874636],
        [ 0.23769031],
        [ 0.23955296],
        ...,
        [10.        ],
        [11.        ],
        [12.        ]],

       [[ 0.23769031],
        [ 0.23955296],
        [ 0.24481697],
        ...,
        [11.        ],
        [12.        ],
        [13.        ]],

       ...,

       [[ 0.33422417],
        [ 0.30774214],
        [ 0.29462261],
        ...,
        [10.        ],
        [11.        ],
        [12.        ]],

       [[ 0.30774214],
        [ 0.29462261],
        [ 0.31689342],
        ...,
        [11.        ],
        [12.        ],
        [13.        ]],

       [[ 0.29462261],
        [ 0.31689342],
        [ 0.3819242 ],
        ...,
        [12.        ],
        [13.        ],
        [14.        ]]])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(demand_X, y, test_size=0.33, random_state=42)

In [17]:
# Initialising the RNN
my_regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
my_regressor.add(LSTM(units = 30, return_sequences = True,
                      input_shape = (demand_X.shape[1], 1)))
my_regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
#my_regressor.add(LSTM(units = 50, return_sequences = True))
#my_regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
#my_regressor.add(LSTM(units = 50, return_sequences = True))
#my_regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
my_regressor.add(LSTM(units = 30))
my_regressor.add(Dropout(0.2))

# Adding the output layer
my_regressor.add(Dense(units = 1))

# Compiling the RNN
my_regressor.compile(optimizer = 'adam', loss = 'mean_squared_error',
                     metrics = ['mean_squared_error'])

Metal device set to: Apple M1 Pro


2022-01-18 20:48:40.218529: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-18 20:48:40.219289: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [18]:
# Fitting the RNN to the Training set
history = my_regressor.fit(X_train,
                           y_train,
                           validation_split=0.2,
                           epochs = 100,
                           batch_size = 32)

2022-01-18 20:48:53.019992: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100


2022-01-18 20:48:53.760563: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-18 20:48:53.994323: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-18 20:48:54.112271: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-18 20:48:54.249512: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-18 20:48:54.441181: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


16/16 [==============================] - ETA: 0s - loss: 0.1632 - mean_squared_error: 0.1632

2022-01-18 20:48:55.764396: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-18 20:48:55.842445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-18 20:48:55.921027: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


16/16 [==============================] - 3s 104ms/step - loss: 0.1632 - mean_squared_error: 0.1632 - val_loss: 0.0747 - val_mean_squared_error: 0.0747
Epoch 2/100
16/16 [==============================] - 1s 42ms/step - loss: 0.0453 - mean_squared_error: 0.0453 - val_loss: 0.0336 - val_mean_squared_error: 0.0336
Epoch 3/100
16/16 [==============================] - 1s 41ms/step - loss: 0.0370 - mean_squared_error: 0.0370 - val_loss: 0.0320 - val_mean_squared_error: 0.0320
Epoch 4/100
16/16 [==============================] - 1s 42ms/step - loss: 0.0652 - mean_squared_error: 0.0652 - val_loss: 0.0466 - val_mean_squared_error: 0.0466
Epoch 5/100
16/16 [==============================] - 1s 41ms/step - loss: 0.0848 - mean_squared_error: 0.0848 - val_loss: 0.0527 - val_mean_squared_error: 0.0527
Epoch 6/100
16/16 [==============================] - 1s 40ms/step - loss: 0.0778 - mean_squared_error: 0.0778 - val_loss: 0.0576 - val_mean_squared_error: 0.0576
Epoch 7/100
16/16 [====================

In [20]:
y_pred_demand = my_regressor.predict(X_test)

2022-01-18 20:51:57.197738: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-18 20:51:57.261193: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-18 20:51:57.338142: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [22]:
mse = mean_absolute_error(y_test, y_pred_demand)
mse

0.17847140606453804

In [23]:
X_train, X_test, y_train, y_test = train_test_split(demand_hours_X, y, test_size=0.33, random_state=42)

In [30]:
# Initialising the RNN
my_regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
my_regressor.add(LSTM(units = 30, return_sequences = True,
                      input_shape = (demand_X.shape[1], 1)))
my_regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
#my_regressor.add(LSTM(units = 50, return_sequences = True))
#my_regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
#my_regressor.add(LSTM(units = 50, return_sequences = True))
#my_regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
my_regressor.add(LSTM(units = 30))
my_regressor.add(Dropout(0.2))

# Adding the output layer
my_regressor.add(Dense(units = 1))

# Compiling the RNN
my_regressor.compile(optimizer = 'adam', loss = 'mean_squared_error',
                     metrics = ['mean_squared_error'])

In [28]:
# Fitting the RNN to the Training set
history = my_regressor.fit(X_train,
                           y_train,
                           validation_split=0.2,
                           epochs = 100,
                           batch_size = 32)

Epoch 1/100


ValueError: in user code:

    File "/Users/daniyarkurmanbayev/miniforge3/envs/globalenv/lib/python3.9/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/Users/daniyarkurmanbayev/miniforge3/envs/globalenv/lib/python3.9/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/daniyarkurmanbayev/miniforge3/envs/globalenv/lib/python3.9/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/Users/daniyarkurmanbayev/miniforge3/envs/globalenv/lib/python3.9/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/Users/daniyarkurmanbayev/miniforge3/envs/globalenv/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) fr

In [26]:
y_pred_demand = my_regressor.predict(X_test)

ValueError: in user code:

    File "/Users/daniyarkurmanbayev/miniforge3/envs/globalenv/lib/python3.9/site-packages/keras/engine/training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "/Users/daniyarkurmanbayev/miniforge3/envs/globalenv/lib/python3.9/site-packages/keras/engine/training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/daniyarkurmanbayev/miniforge3/envs/globalenv/lib/python3.9/site-packages/keras/engine/training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/daniyarkurmanbayev/miniforge3/envs/globalenv/lib/python3.9/site-packages/keras/engine/training.py", line 1572, in predict_step
        return self(x, training=False)
    File "/Users/daniyarkurmanbayev/miniforge3/envs/globalenv/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filter

In [22]:
mse = mean_absolute_error(y_test, y_pred_demand)
mse

0.17847140606453804